Instalação das bibliotecas necessárioas

In [ ]:
!python -m pip install pymongo
!python -m pip install PyMySQL
!python -m pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.1 MB/s eta 0:00:00


Import das bibliotecas

In [ ]:
from pymongo import MongoClient
import pymysql
from faker import Faker
from faker.providers import automotive
import time
import random
from datetime import datetime, timedelta, timezone

# Nessita para ser utilizado:
# Banco da dados MongoDB: ['dbname']
# Collections: ['telemetrias', 'veiculos']
# Banco da dados MySQL: ['dbname']
# SQL disponível no repositório Github: https://github.com/RechargeCG/projeto-banco-de-dados-iot.git

uri = 'endereço_do_banco_de_dados_mongodb'
client = MongoClient( uri )
mydb = client["dbname"]

print(mydb.list_collection_names())

mysqldb = pymysql.connect(
   host="host",
   user="user",
   password="password",
   db="dbname",
   charset="utf8"
)

# print(mysqldb)
#criando o cursor
mycursor = mysqldb.cursor()

mycursor.execute("show tables") #aqui o ; é opcional

#exibindo as informações
for x in mycursor:
  print(x)

['veiculos', 'telemetrias']
('Comodo',)
('Configuracao_Sensores',)
('Diagnostico_Sensores',)
('Historico',)
('Lampada',)
('Morador',)
('Reside',)
('Residencia',)
('Telemetrias',)
('Veiculos',)


Criação dos dados

In [ ]:
# Seed para garantir dados consistentes entre testes
seed = 13
random.seed(seed)

# Inicializa Faker
Faker.seed(seed)
fake = Faker()
fake.add_provider(automotive)

# Funções auxiliares para dados realistas dos veículos
def gerar_sensores_config():
    sensores_base = [
        # Visão Principal e Profundidade
        {'tipo': 'LiDAR_360', 'versao_hw': 'V4.5', 'local': 'Teto', 'ult_calibracao': fake.date_time_between(start_date='-6m', end_date='now', tzinfo=None)},
        {'tipo': 'Radar_Frontal', 'versao_hw': 'V5.1', 'local': 'Parachoque', 'ult_calibracao': fake.date_time_between(start_date='-3m', end_date='now', tzinfo=None)},
        {'tipo': 'Camera_Estereo', 'versao_hw': 'V10.2', 'local': 'Parabrisa', 'ult_calibracao': fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None)},
        {'tipo': 'Radar_Lateral_Dir', 'versao_hw': 'V5.1', 'local': 'Lateral_Dir', 'ult_calibracao': fake.date_time_between(start_date='-3m', end_date='now', tzinfo=None)},
        # Localização e odometria
        {'tipo': 'Receptor_GPS_RTK', 'versao_hw': 'V2.2', 'local': 'Antena_Teto', 'ult_calibracao': fake.date_time_between(start_date='-2y', end_date='now', tzinfo=None)},
        {'tipo': 'IMU', 'versao_hw': 'V8.0', 'local': 'Chassi_Central', 'ult_calibracao': fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None)},
        # Sistemas de controle
        {'tipo': 'Sensor_Torque_Volante', 'versao_hw': 'V1.1', 'local': 'Coluna_Direcao', 'ult_calibracao': fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None)},
        {'tipo': 'Sensor_Pressao_Freio', 'versao_hw': 'V2.0', 'local': 'Unid_Hidraulica', 'ult_calibracao': fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None)}
    ]
    return sensores_base

def gerar_diagnostico_sensores(sensores_config):
    diagnosticos = []
    for sensor in sensores_config:
        status = 'OK'
        # Gera valores base para OK
        taxa_erros = round(random.uniform(0.001, 0.1), 3)
        temp = round(random.uniform(30.0, 50.0), 1)
        cod_falha = None

        if random.random() < 0.05:
            status = 'Degradado'
            taxa_erros = round(random.uniform(0.1, 0.9), 3)
            temp = round(random.uniform(50.0, 75.0), 1)

        elif random.random() < 0.02:
            status = 'Falha'
            taxa_erros = 1.0
            temp = 99.9

            tipo = sensor['tipo']

            if 'Camera' in tipo:
                cod_falha = random.choice(['CAM_E01', 'CAM_L04', 'CAM_S02'])
            elif 'LiDAR' in tipo:
                cod_falha = random.choice(['LiDAR_D02', 'LiDAR_S01', 'LiDAR_T03'])
            elif 'Radar' in tipo:
                cod_falha = random.choice(['RAD_S05', 'RAD_T01', 'RAD_F07'])
            elif 'GPS' in tipo:
                cod_falha = random.choice(['GPS_N01', 'GPS_T01'])
            elif 'IMU' in tipo:
                cod_falha = random.choice(['IMU_F03', 'IMU_G01'])
            elif 'Sensor' in tipo:
                cod_falha = random.choice(['ACT_C01', 'ACT_M02'])
            else:
                cod_falha = 'GER_U99'

        diagnostico = {
            'nome_sensor': sensor['tipo'],
            'status': status,
            'taxa_erros_pct': taxa_erros,
            'temp_operacao_c': temp,
            'cod_falha': cod_falha
        }
        diagnosticos.append(diagnostico)
    return diagnosticos

# Gerando dados base
marca_modelo = {}
for i in range(100):
  marca = fake.last_name()
  marca_modelo[i] = [marca, []]
  for j in range(30):
    modelo = fake.last_name()
    marca_modelo[i][1].append(modelo)

tipos_propulsao = ['Eletrico', 'Combustao', 'Hibrido']

num_veiculos = 5000
num_leituras = 50000

# Gerando veículos
veiculos_autonomos = []
config_map = {}

for i in range(num_veiculos):
    marcaemodelo = random.choice(marca_modelo)
    marca = marcaemodelo[0]
    modelo = random.choice(marcaemodelo[1])

    vin = fake.vin()
    config = gerar_sensores_config()
    tipo_propulsao = random.choice(tipos_propulsao)

    veiculo = {
        '_id': vin,
        'marca': marca,
        'modelo': modelo,
        'nivel_autonomia': random.randint(3, 5),
        'tipo_propulsao': tipo_propulsao,
        'data_de_fabricacao': fake.date_time_between(start_date='-5y', end_date='now', tzinfo=None),
        'sensores_config': config
    }

    veiculos_autonomos.append(veiculo)
    config_map[vin] = {'config': config, 'propulsao': tipo_propulsao}


# Gerando telemetrias
telemetrias = []

for i in range(num_leituras):
    veiculo_idx = random.randint(0, len(veiculos_autonomos) - 1)
    id_veiculo = veiculos_autonomos[veiculo_idx]['_id']

    veiculo_data = config_map[id_veiculo]
    config_base = veiculo_data['config']
    propulsao_do_veiculo = veiculo_data['propulsao']

    leitura = {
        'id_veiculo': id_veiculo,
        'tempo_ativo_segundos': random.randint(0,100000),
        'timestamp': fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None),
        'quilometragem': random.randint(0,999999),
        'velocidade_kmh': random.randint(0,180),
        'diagnostico_sensores': gerar_diagnostico_sensores(config_base)
    }

    if propulsao_do_veiculo == 'Eletrico' or propulsao_do_veiculo == 'Hibrido':
        leitura['nivel_bateria_percentual'] = random.randint(0, 100)
    if propulsao_do_veiculo == 'Combustao' or propulsao_do_veiculo == 'Hibrido':
        leitura['nivel_combustivel_percentual'] = random.randint(0, 100)

    telemetrias.append(leitura)

Etapa de inserção MongoDB

In [ ]:
################################################################################
#                   Inserção no MongoDB - com marcação de tempo                #
################################################################################

mycol_veiculos = mydb["veiculos"]
mycol_telemetrias = mydb["telemetrias"]
mycol_veiculos.delete_many({})
mycol_telemetrias.delete_many({})
print("Dados do banco removidos.")

############################### INSERT VEICULOS ###############################

# Marca o início
inicio = time.time()
#realiza o insert
mycol_veiculos.insert_many(veiculos_autonomos)
# Marca o fim
fim = time.time()
# Tempo
tempo_execucao = fim - inicio
print(f"Inserção na Collection veiculos de {len(veiculos_autonomos)} documentos durou {tempo_execucao:.4f} segundos")


############################### INSERT TELEMETRIAS ###############################

# Marca o início
inicio = time.time()
#realiza o insert
# mycol_telemetrias.insert_many(telemetrias)
batch_size = 5000
for i in range(0, len(telemetrias), batch_size):
    batch = telemetrias[i:i + batch_size]
    mycol_telemetrias.insert_many(batch)
    print(f"Lote {i//batch_size + 1} de {len(telemetrias)//batch_size} inserido.")
# Marca o fim
fim = time.time()
# Tempo
tempo_execucao = fim - inicio
print(f"Inserção na Collection devices de {len(telemetrias)} documentos durou {tempo_execucao:.4f} segundos")

Dados do banco removidos.
Inserção na Collection veiculos de 5000 documentos durou 7.8034 segundos
Lote 1 de 10 inserido.
Lote 2 de 10 inserido.
Lote 3 de 10 inserido.
Lote 4 de 10 inserido.
Lote 5 de 10 inserido.
Lote 6 de 10 inserido.
Lote 7 de 10 inserido.
Lote 8 de 10 inserido.
Lote 9 de 10 inserido.
Lote 10 de 10 inserido.
Inserção na Collection devices de 50000 documentos durou 74.3530 segundos


Etapa de inserção MySQL

In [ ]:
def normalizar_para_mysql(veiculos_autonomos, telemetrias):
    sql_veiculos = []
    sql_configuracao_sensores = []
    sql_telemetria = []
    sql_diagnostico_sensores = []

    leitura_id_counter = 1
    config_id_counter = 1
    diagnostico_id_counter = 1

    # ====================================================================
    # A) PROCESSAMENTO DA COLEÇÃO VEICULOS
    # ====================================================================

    for veiculo in veiculos_autonomos:
        vin = veiculo['_id']

        # 1. Tabela VEICULOS
        sql_veiculos.append({
            'veiculo_id': vin,
            'marca': veiculo['marca'],
            'modelo': veiculo['modelo'],
            'nivel_autonomia': veiculo['nivel_autonomia'],
            'tipo_propulsao': veiculo['tipo_propulsao'],
            'data_fabricacao': veiculo['data_de_fabricacao'].strftime('%Y-%m-%d')
        })

        # 2. Tabela CONFIGURACAO_SENSORES
        for sensor_config in veiculo['sensores_config']:
            sql_configuracao_sensores.append({
                'config_id': config_id_counter,
                'fk_Veiculos_veiculo_id': vin,
                'nome_sensor': sensor_config['tipo'],
                'versao_hardware': sensor_config['versao_hw'],
                'ultima_calibracao': sensor_config['ult_calibracao'].strftime('%Y-%m-%d %H:%M:%S')
            })
            config_id_counter += 1

    # ====================================================================
    # B) PROCESSAMENTO DA COLEÇÃO TELEMETRIAS
    # ====================================================================

    for leitura in telemetrias:
        vin = leitura['id_veiculo']
        current_leitura_id = leitura_id_counter

        # 1. Tabela TELEMETRIA (CORRIGIDA: tempo_ativo_segundos adicionado)
        sql_telemetria.append({
            'leitura_id': current_leitura_id,
            'fk_Veiculos_veiculo_id': vin,
            'timestamp': leitura['timestamp'].strftime('%Y-%m-%d %H:%M:%S'),
            'tempo_ativo_segundos': leitura.get('tempo_ativo_segundos'), # Adicionado get() para garantir que sempre pega o valor
            'quilometragem_km': leitura['quilometragem'],
            'velocidade_kph': leitura['velocidade_kmh'],
            'nivel_bateria_pct': leitura.get('nivel_bateria_percentual'),
            'nivel_combustivel_pct': leitura.get('nivel_combustivel_percentual')
        })
        leitura_id_counter += 1

        # 2. Tabela DIAGNOSTICO_SENSORES (CORRIGIDA: temp_operacao_c adicionado)
        for diagnostico in leitura['diagnostico_sensores']:
            sql_diagnostico_sensores.append({
                'diagnostico_id': diagnostico_id_counter,
                'fk_Telemetrias_leitura_id': current_leitura_id,
                'nome_sensor': diagnostico['nome_sensor'],
                'status_sensor': diagnostico['status'],
                'taxa_erros_pct': diagnostico['taxa_erros_pct'],
                'temp_operacao_c': diagnostico['temp_operacao_c'], # CAMPO CRUCIAL ADICIONADO
                'cod_falha': diagnostico['cod_falha']
            })
            diagnostico_id_counter += 1

    print(f"Normalização concluída: {len(sql_veiculos)} Veículos, {len(sql_telemetria)} Telemetrias, {len(sql_configuracao_sensores)} Configurações, {len(sql_diagnostico_sensores)} Diagnósticos.")
    return sql_veiculos, sql_configuracao_sensores, sql_telemetria, sql_diagnostico_sensores

def inserir_em_lote_mysql(nome_tabela, lista_dados):
    if not lista_dados:
        print(f"Lista de dados para {nome_tabela} está vazia. Pulando.")
        return

    # Extrai os nomes das colunas da primeira entrada (todos os dicts devem ser iguais)
    colunas = lista_dados[0].keys()

    # Monta a string de colunas e placeholders (%s)
    colunas_sql = ", ".join(colunas)
    placeholders = ", ".join(["%s"] * len(colunas))

    # Query final: Ex: INSERT INTO Veiculos (veiculo_id, marca, modelo, ...) VALUES (%s, %s, %s, ...)
    query = f"INSERT INTO {nome_tabela} ({colunas_sql}) VALUES ({placeholders})"

    # Prepara os dados para o executemany (transforma lista de dicts em lista de tuplas)
    dados_para_insercao = [tuple(d.values()) for d in lista_dados]

    print(f"Inserindo {len(dados_para_insercao)} linhas na tabela {nome_tabela}...")

    return query, dados_para_insercao
    # O comando executemany() do Python é o equivalente ao 'insert_many' do MongoDB
    # cursor.executemany(query, dados_para_insercao)
    # print(f"Inserção em {nome_tabela} concluída com sucesso.")

res = normalizar_para_mysql(veiculos_autonomos,telemetrias)

################################################################################
#                   Inserção no MySQL - com marcação de tempo                #
################################################################################

mycursor.execute("TRUNCATE TABLE Diagnostico_Sensores")
mycursor.execute("TRUNCATE TABLE Telemetrias")
mycursor.execute("TRUNCATE TABLE Configuracao_Sensores")
mycursor.execute("TRUNCATE TABLE Veiculos")
mysqldb.commit()
print("Dados do banco removidos.")

############################### INSERT VEICULOS ###############################

query, dados_para_insercao = inserir_em_lote_mysql("Veiculos", res[0])
# Marca o início
inicio = time.time()
#realiza o insert
mycursor.executemany(query, dados_para_insercao)
mysqldb.commit()
# Marca o fim
fim = time.time()
# Tempo
tempo_execucao = fim - inicio
print(f"Inserção na tabela Veiculos de {len(res[0])} documentos durou {tempo_execucao:.4f} segundos")

############################### INSERT CONFIGURACOES_SENSORES ###############################

query, dados_para_insercao = inserir_em_lote_mysql("Configuracao_Sensores", res[1])
# Marca o início
inicio = time.time()
#realiza o insert
mycursor.executemany(query, dados_para_insercao)
mysqldb.commit()
# Marca o fim
fim = time.time()
# Tempo
tempo_execucao = fim - inicio
print(f"Inserção na tabela Configuracao_Sensores de {len(res[1])} documentos durou {tempo_execucao:.4f} segundos")

############################### INSERT TELEMETRIAS ###############################

query, dados_para_insercao = inserir_em_lote_mysql("Telemetrias", res[2])
# Marca o início
inicio = time.time()
#realiza o insert
mycursor.executemany(query, dados_para_insercao)
mysqldb.commit()
# Marca o fim
fim = time.time()
# Tempo
tempo_execucao = fim - inicio
print(f"Inserção na tabela Telemetrias de {len(res[2])} documentos durou {tempo_execucao:.4f} segundos")

############################### INSERT DIAGNOSTICO_SENSORES ###############################

query, dados_para_insercao = inserir_em_lote_mysql("Diagnostico_Sensores", res[3])
# Marca o início
inicio = time.time()
#realiza o insert
mycursor.executemany(query, dados_para_insercao)
mysqldb.commit()
# Marca o fim
fim = time.time()
# Tempo
tempo_execucao = fim - inicio
print(f"Inserção na tabela Diagnostico_Sensores de {len(res[3])} documentos durou {tempo_execucao:.4f} segundos")

Normalização concluída: 5000 Veículos, 50000 Telemetrias, 40000 Configurações, 400000 Diagnósticos.
Dados do banco removidos.
Inserindo 5000 linhas na tabela Veiculos...
Inserção na tabela Veiculos de 5000 documentos durou 1.6677 segundos
Inserindo 40000 linhas na tabela Configuracao_Sensores...
Inserção na tabela Configuracao_Sensores de 40000 documentos durou 2.3434 segundos
Inserindo 50000 linhas na tabela Telemetrias...
Inserção na tabela Telemetrias de 50000 documentos durou 2.2330 segundos
Inserindo 400000 linhas na tabela Diagnostico_Sensores...
Inserção na tabela Diagnostico_Sensores de 400000 documentos durou 20.9489 segundos


Etapa de buscas MongoDB


In [ ]:
# Qual é a taxa de erro média dos sensores LiDAR instalados em veículos de Nível SAE 4
# que estão operando em velocidade superior a 80 km/h e qual a porcentagem desses sensores
# que atingiu o status "Degradado" na última hora?
mycol_telemetrias = mydb['telemetrias']
inicio = time.time()
res = (list(mycol_telemetrias.aggregate([
    {
        '$match': {
            'timestamp': { '$gte': datetime.now() - timedelta(hours=1) }
        }
    },
    {
        '$lookup': {
            'from': 'veiculos',
            'localField': 'id_veiculo',
            'foreignField': '_id',
            'as': 'dados_veiculo'
        }
    },
    {
        '$unwind': '$dados_veiculo'
    },
    {
        '$match': {
            'dados_veiculo.nivel_autonomia': 4,
            'velocidade_kmh': { '$gt': 80 }
        }
    },
    {
        '$unwind': '$diagnostico_sensores'
    },
    {
        '$match': {
            'diagnostico_sensores.nome_sensor': 'LiDAR_360'
        }
    },
    {
        '$group': {
            '_id': None,
            'taxa_erro_media': { '$avg': '$diagnostico_sensores.taxa_erros_pct' },
            'total_sensores': { '$sum': 1 },
            'sensores_degradados': {
                '$sum': {
                    '$cond': [ { '$eq': ['$diagnostico_sensores.status', 'Degradado'] }, 1, 0 ]
                }
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'taxa_erro_media': { '$round': ['$taxa_erro_media', 4] },
            'porcentagem_degradada': {
                '$round': [
                    { '$multiply': [ { '$divide': ['$sensores_degradados', '$total_sensores'] }, 100 ] }, 2
                ]
            }
        }
    }
])))
fim = time.time()
tempo_execucao = fim - inicio
print(f"Taxa de erro media LiDAR SAE 4 > 80km/h: (Tempo: {tempo_execucao:.4f} segundos)")
print(res)

# ---------------------------------------------------------------------------

# Qual a quantidade total de veículos que tiveram qualquer sensor LiDAR com erro alto (> 0.1%)
# nas últimas 24 horas?
mycol_telemetrias = mydb['telemetrias']
inicio = time.time()
res = (list(mycol_telemetrias.aggregate([
    {
        '$match': {
            'timestamp': { '$gte': datetime.now() - timedelta(hours=24) }
        }
    },
    {
        '$unwind': '$diagnostico_sensores'
    },
    {
        '$match': {
            'diagnostico_sensores.nome_sensor': 'LiDAR_360',
            'diagnostico_sensores.taxa_erros_pct': { '$gt': 0.1 }
        }
    },
    {
        '$group': {
            '_id': '$id_veiculo'
        }
    },
    {
        '$group': {
            '_id': None,
            'total_veiculos': { '$sum': 1 }
        }
    },
    {
        '$project': {
            '_id': 0,
            'total_veiculos': 1
        }
    }
])))
fim = time.time()
tempo_execucao = fim - inicio
print(f"Total de veículos (LiDAR com erro alto, última 24h): (Tempo: {tempo_execucao:.4f} segundos)")
print(res)

# ---------------------------------------------------------------------------

# Dado um código de falha crítico (CAM_E01), qual é o VIN do veículo, o modelo da 
# leitura onde a falha foi registrada pela primeira vez, e há quanto tempo o veículo
# estava em operação (tempo_ativo_segundos) quando isso ocorreu?
mycol_telemetrias = mydb['telemetrias']
inicio = time.time()
res = (list(mycol_telemetrias.aggregate([
    {
        '$unwind': '$diagnostico_sensores'
    },
    {
        '$match': {
            'diagnostico_sensores.cod_falha': 'CAM_E01'
        }
    },
    {
        '$sort': {
            'timestamp': 1
        }
    },
    {
        '$limit': 1
    },
    {
        '$lookup': {
            'from': 'veiculos',
            'localField': 'id_veiculo',
            'foreignField': '_id',
            'as': 'dados_veiculo'
        }
    },
    {
        '$unwind': '$dados_veiculo'
    },
    {
        '$project': {
            '_id': 0,
            'VIN': '$id_veiculo',
            'modelo_veiculo': '$dados_veiculo.modelo',
            'tempo_ativo_segundos': '$tempo_ativo_segundos',
            'data_falha': '$timestamp'
        }
    }
])))
fim = time.time()
tempo_execucao = fim - inicio
print(f"Dados da primeira falha crítica (CAM_E01): (Tempo: {tempo_execucao:.4f} segundos)")
print(res)

# ---------------------------------------------------------------------------

# Qual é a Taxa de Falha Crítica por Quilometragem Rodada (Falhas/1000 km)
# para veículos fabricados antes de 2023-01-01, comparada aos modelos fabricados posteriormente?
mycol_telemetrias = mydb['telemetrias']
inicio = time.time()
res_nova = (list(mycol_telemetrias.aggregate([
    {
        '$lookup': {
            'from': 'veiculos',
            'localField': 'id_veiculo',
            'foreignField': '_id',
            'as': 'dados_veiculo'
        }
    },
    {
        '$unwind': '$dados_veiculo'
    },
    {
        '$unwind': '$diagnostico_sensores'
    },
    {
        '$group': {
            '_id': '$id_veiculo',
            'data_fabricacao': { '$first': '$dados_veiculo.data_de_fabricacao' },
            'quilometragem': { '$max': '$quilometragem' },
            'total_falhas_criticas': {
                '$sum': {
                    '$cond': [ { '$eq': ['$diagnostico_sensores.status', 'Falha'] }, 1, 0 ]
                }
            }
        }
    },
    {
        '$group': {
            '_id': {
                '$lt': ['$data_fabricacao', datetime(2023, 1, 1)]
            },
            'total_km_rodado': { '$sum': '$quilometragem' },
            'soma_total_falhas': { '$sum': '$total_falhas_criticas' }
        }
    },
    {
        '$project': {
            '_id': 0,
            'grupo': {
                '$cond': [ '$_id', 'Frota Antiga (Antes de 2023)', 'Frota Nova (2023 em diante)' ]
            },
            'falhas_por_1000km': {
                '$round': [
                    { '$multiply': [ { '$divide': ['$soma_total_falhas', '$total_km_rodado'] }, 1000 ] }, 4
                ]
            }
        }
    }
])))
fim = time.time()
tempo_execucao = fim - inicio
print(f"Taxa de falha crítica por 1000 km (comparativo por idade): (Tempo: {tempo_execucao:.4f} segundos)")
print(res_nova)

# ---------------------------------------------------------------------------

# Quais são os 5 sensores que mais frequentemente reportam temperatura de operação acima de 70°C
# e em que marcas de veículos esses sensores estão predominantemente instalados?
mycol_telemetrias = mydb['telemetrias']
inicio = time.time()
res = (list(mycol_telemetrias.aggregate([
    {
        '$unwind': '$diagnostico_sensores'
    },
    {
        '$match': {
            'diagnostico_sensores.temp_operacao_c': { '$gt': 70.0 }
        }
    },
    {
        '$lookup': {
            'from': 'veiculos',
            'localField': 'id_veiculo',
            'foreignField': '_id',
            'as': 'dados_veiculo'
        }
    },
    {
        '$unwind': '$dados_veiculo'
    },
    {
        '$group': {
            '_id': {
                'sensor': '$diagnostico_sensores.nome_sensor',
                'marca': '$dados_veiculo.marca'
            },
            'frequencia_superaquecimento': { '$sum': 1 }
        }
    },
    {
        '$sort': {
            'frequencia_superaquecimento': -1
        }
    },
    {
        '$limit': 5
    },
    {
        '$project': {
            '_id': 0,
            'sensor': '$_id.sensor',
            'marca_veiculo': '$_id.marca',
            'contagem': '$frequencia_superaquecimento'
        }
    }
])))
fim = time.time()
tempo_execucao = fim - inicio
print(f"Top 5 sensores com superaquecimento por marca: (Tempo: {tempo_execucao:.4f} segundos)")
print(res)

Taxa de erro media LiDAR SAE 4 > 80km/h: (Tempo: 0.4456 segundos)
[{'taxa_erro_media': 0.046, 'porcentagem_degradada': 0.0}]
Total de veículos (LiDAR com erro alto, última 24h): (Tempo: 0.3734 segundos)
[{'total_veiculos': 10}]
Dados da primeira falha crítica (CAM_E01): (Tempo: 0.7382 segundos)
[{'VIN': '9XKWL4VG09U9R3638', 'modelo_veiculo': 'Hall', 'tempo_ativo_segundos': 78111, 'data_falha': datetime.datetime(2024, 12, 10, 6, 39, 52, 655000)}]
Taxa de falha crítica por 1000 km (comparativo por idade): (Tempo: 3.2175 segundos)
[{'grupo': 'Frota Antiga (Antes de 2023)', 'falhas_por_1000km': 0.0017}, {'grupo': 'Frota Nova (2023 em diante)', 'falhas_por_1000km': 0.0017}]
Top 5 sensores com superaquecimento por marca: (Tempo: 1.3531 segundos)
[{'sensor': 'Radar_Frontal', 'marca_veiculo': 'Williams', 'contagem': 69}, {'sensor': 'Receptor_GPS_RTK', 'marca_veiculo': 'Williams', 'contagem': 66}, {'sensor': 'Radar_Lateral_Dir', 'marca_veiculo': 'Williams', 'contagem': 65}, {'sensor': 'Camera_E

Etapa de buscas MySQL

In [ ]:
mycursor = mysqldb.cursor()
# Assumindo que 'mycursor' e 'mydb' estão definidos e a conexão está ativa.

# A data de corte de 2023-01-01 deve ser passada como string no SQL.
DATA_CORTE_FROTA_ANTIGA = '2023-01-01'

# Qual é a taxa de erro média dos sensores LiDAR instalados em veículos de Nível SAE 4
# que estão operando em velocidade superior a 80 km/h e qual a porcentagem desses sensores
# que atingiu o status "Degradado" na última hora?
inicio = time.time()
query_sql_1 = """
SELECT
    ROUND(AVG(d.taxa_erros_pct), 4) AS taxa_erro_media,
    ROUND(SUM(CASE WHEN d.status_sensor = 'Degradado' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS porcentagem_degradada
FROM Telemetrias t
JOIN Veiculos v ON t.fk_Veiculos_veiculo_id = v.veiculo_id
JOIN Diagnostico_Sensores d ON t.leitura_id = d.fk_Telemetrias_leitura_id
WHERE
    t.timestamp >= DATE_SUB(NOW(), INTERVAL 1 HOUR) AND
    v.nivel_autonomia = 4 AND
    t.velocidade_kph > 80 AND
    d.nome_sensor = 'LiDAR_360';
"""
mycursor.execute(query_sql_1)
res = mycursor.fetchall()
fim = time.time()
tempo_execucao = fim - inicio
print(f"Taxa de erro media LiDAR SAE 4 > 80km/h: (Tempo: {tempo_execucao:.4f} segundos)")
print(res)

# ---------------------------------------------------------------------------

# Qual a quantidade total de veículos que tiveram qualquer sensor LiDAR com erro alto (> 0.1%)
# nas últimas 24 horas?
inicio = time.time()
query_sql_2 = """
SELECT
    COUNT(DISTINCT t.fk_Veiculos_veiculo_id) AS total_veiculos
FROM Telemetrias t
JOIN Diagnostico_Sensores d ON t.leitura_id = d.fk_Telemetrias_leitura_id
WHERE
    t.timestamp >= DATE_SUB(NOW(), INTERVAL 24 HOUR) AND
    d.nome_sensor = 'LiDAR_360' AND
    d.taxa_erros_pct > 0.1;
"""
mycursor.execute(query_sql_2)
res = mycursor.fetchall()
fim = time.time()
tempo_execucao = fim - inicio
print(f"Total de veículos (LiDAR com erro alto, última 24h): (Tempo: {tempo_execucao:.4f} segundos)")
print(res)

# ---------------------------------------------------------------------------

# Dado um código de falha crítico (CAM_E01), qual é o VIN do veículo, o modelo da 
# leitura onde a falha foi registrada pela primeira vez, e há quanto tempo o veículo
# estava em operação (tempo_ativo_segundos) quando isso ocorreu?

inicio = time.time()
query_sql_3 = """
SELECT
    v.veiculo_id AS VIN,
    v.modelo,
    t.tempo_ativo_segundos,
    t.timestamp AS data_falha
FROM Telemetrias t
JOIN Veiculos v ON t.fk_Veiculos_veiculo_id = v.veiculo_id
JOIN Diagnostico_Sensores d ON t.leitura_id = d.fk_Telemetrias_leitura_id
WHERE
    d.cod_falha = 'CAM_E01'
ORDER BY
    t.timestamp ASC
LIMIT 1;
"""
mycursor.execute(query_sql_3)
res = mycursor.fetchall()
fim = time.time()
tempo_execucao = fim - inicio
print(f"Dados da primeira falha crítica (CAM_E01): (Tempo: {tempo_execucao:.4f} segundos)")
print(res)

# ---------------------------------------------------------------------------

# Qual é a Taxa de Falha Crítica por Quilometragem Rodada (Falhas/1000 km)
# para veículos fabricados antes de 2023-01-01, comparada aos modelos fabricados posteriormente?
inicio = time.time()
query_sql_4 = f"""
SELECT
    CASE
        WHEN V.data_fabricacao < '{DATA_CORTE_FROTA_ANTIGA}' THEN 'Frota Antiga (Antes de 2023)'
        ELSE 'Frota Nova (2023 em diante)'
    END AS grupo,
    ROUND(SUM(T1.total_falhas_criticas) * 1000.0 / SUM(T1.quilometragem_max), 4) AS falhas_por_1000km
FROM Veiculos V
JOIN (
    -- Subquery T1: Resumo de Falhas e Max Kilometragem por Veiculo
    SELECT
        T.fk_Veiculos_veiculo_id,
        MAX(T.quilometragem_km) AS quilometragem_max,
        SUM(CASE WHEN D.status_sensor = 'Falha' THEN 1 ELSE 0 END) AS total_falhas_criticas
    FROM Telemetrias T
    JOIN Diagnostico_Sensores D ON T.leitura_id = D.fk_Telemetrias_leitura_id
    GROUP BY T.fk_Veiculos_veiculo_id
) T1 ON V.veiculo_id = T1.fk_Veiculos_veiculo_id
GROUP BY grupo;
"""
mycursor.execute(query_sql_4)
res_nova = mycursor.fetchall()
fim = time.time()
tempo_execucao = fim - inicio
print(f"Taxa de falha crítica por 1000 km (comparativo por idade): (Tempo: {tempo_execucao:.4f} segundos)")
print(res_nova)

# ---------------------------------------------------------------------------

# Quais são os 5 sensores que mais frequentemente reportam temperatura de operação acima de 70°C
# e em que marcas de veículos esses sensores estão predominantemente instalados?
inicio = time.time()
query_sql_5 = """
SELECT
    d.nome_sensor AS sensor,
    v.marca AS marca_veiculo,
    COUNT(*) AS contagem
FROM Diagnostico_Sensores d
JOIN Telemetrias t ON d.fk_Telemetrias_leitura_id = t.leitura_id
JOIN Veiculos v ON t.fk_Veiculos_veiculo_id = v.veiculo_id
WHERE
    d.temp_operacao_c > 70.0
GROUP BY
    d.nome_sensor, v.marca
ORDER BY
    contagem DESC
LIMIT 5;
"""
mycursor.execute(query_sql_5)
res = mycursor.fetchall()
fim = time.time()
tempo_execucao = fim - inicio
print(f"Top 5 sensores com superaquecimento por marca: (Tempo: {tempo_execucao:.4f} segundos)")
print(res)

Taxa de erro media LiDAR SAE 4 > 80km/h: (Tempo: 0.3123 segundos)
((Decimal('0.1131'), Decimal('22.22')),)
Total de veículos (LiDAR com erro alto, última 24h): (Tempo: 0.3513 segundos)
((11,),)
Dados da primeira falha crítica (CAM_E01): (Tempo: 1.9621 segundos)
(('9XKWL4VG09U9R3638', 'Hall', 78111, datetime.datetime(2024, 12, 10, 6, 39, 52)),)
Taxa de falha crítica por 1000 km (comparativo por idade): (Tempo: 1.9348 segundos)
(('Frota Antiga (Antes de 2023)', Decimal('0.0017')), ('Frota Nova (2023 em diante)', Decimal('0.0017')))
Top 5 sensores com superaquecimento por marca: (Tempo: 1.7748 segundos)
(('Radar_Frontal', 'Williams', 69), ('Receptor_GPS_RTK', 'Williams', 66), ('Radar_Lateral_Dir', 'Williams', 65), ('Camera_Estereo', 'Green', 62), ('IMU', 'Green', 59))


Etapa de atualizações MongoDB

In [ ]:
from datetime import datetime, timedelta, timezone
import time

# Para todos os veículos de nível de autonomia 3, atualize o nível para 4. (Simulando uma atualização de software)
mycol_veiculos = mydb['veiculos']
inicio = time.time()
res = mycol_veiculos.update_many(
    {
        "nivel_autonomia": 3
    },
    {
        "$set": {
            "nivel_autonomia": 4
        }
    }
)
fim = time.time()
tempo_execucao = fim - inicio
print(f"Atualização do Nível de Autonomia 3 para 4. (Tempo: {tempo_execucao:.4f} segundos)")
print(f"Documentos modificados: {res.modified_count}")

# Atualiza todas as telemetrias da última hora onde o status do LiDAR_360 é 'Degradado'.
mycol_telemetrias = mydb['telemetrias']
inicio = time.time()
data_corte_1_hora = datetime.now(timezone.utc) - timedelta(hours=1)
data_atual_utc = datetime.now(timezone.utc)

res = mycol_telemetrias.update_many(
    {
        "timestamp": {"$gte": data_corte_1_hora},
        "diagnostico_sensores.status": "Degradado",
        "diagnostico_sensores.nome_sensor": "LiDAR_360"
    },
    {
        "$set": {
            "diagnostico_sensores.$[elem].status": "Checked OK",
            "diagnostico_sensores.$[elem].cod_falha": None
        }
    },
    array_filters=[
        {
            "elem.nome_sensor": "LiDAR_360",
            "elem.status": "Degradado"
        }
    ]
)
fim = time.time()
tempo_execucao = fim - inicio
print(f"Correção automatizada de status 'Degradado' em Telemetrias. (Tempo: {tempo_execucao:.4f} segundos)")
print(f"Documentos modificados: {res.modified_count}")

Atualização do Nível de Autonomia 3 para 4. (Tempo: 0.4256 segundos)
Documentos modificados: 1639
Correção automatizada de status 'Degradado' em Telemetrias. (Tempo: 0.5580 segundos)
Documentos modificados: 0


Etapa de atualizações MySQL

In [ ]:
# Para todos os veículos de nível de autonomia 3, atualize o nível para 4. (Simulando uma atualização de software)
inicio = time.time()
query_sql_A = """
UPDATE Veiculos
SET nivel_autonomia = 4
WHERE nivel_autonomia = 3;
"""
mycursor.execute(query_sql_A)
modified_count_A = mycursor.rowcount
mysqldb.commit()

fim = time.time()
tempo_execucao = fim - inicio
print(f"Atualização do Nível de Autonomia 3 para 4. (Tempo: {tempo_execucao:.4f} segundos)")
print(f"Documentos modificados: {modified_count_A}")

# Atualiza todas as telemetrias da última hora onde o status do LiDAR_360 é 'Degradado'.
inicio = time.time()
query_sql_B = """
UPDATE Diagnostico_Sensores d
JOIN Telemetrias t ON d.fk_Telemetrias_leitura_id = t.leitura_id
SET
    d.status_sensor = 'Checked OK',
    d.cod_falha = NULL
WHERE
    t.timestamp >= DATE_SUB(NOW(), INTERVAL 1 HOUR) AND
    d.nome_sensor = 'LiDAR_360' AND
    d.status_sensor = 'Degradado';
"""
mycursor.execute(query_sql_B)
modified_count_B = mycursor.rowcount
mysqldb.commit()

fim = time.time()
tempo_execucao = fim - inicio
print(f"Correção automatizada de status 'Degradado' em Telemetrias. (Tempo: {tempo_execucao:.4f} segundos)")
print(f"Documentos modificados: {modified_count_B}")

Atualização do Nível de Autonomia 3 para 4. (Tempo: 0.4795 segundos)
Documentos modificados: 1639
Correção automatizada de status 'Degradado' em Telemetrias. (Tempo: 1.3354 segundos)
Documentos modificados: 2


Etapa de remoção MongoDB

In [ ]:
leituras_col = mydb['telemetrias']
veiculos_col = mydb['veiculos']

# Remove as leituras feitas a mais de 30 dias na coleção telemetrias. (Simulando uma política de retenção de dados periódicos)
data_corte_30_dias = datetime.now(timezone.utc) - timedelta(days=30)
inicio = time.time()
res_leituras_antigas = leituras_col.delete_many({"timestamp": {"$lt": data_corte_30_dias}})
deleted_leituras_count_antigas = res_leituras_antigas.deleted_count

fim = time.time()
tempo_execucao = fim - inicio
print(f"Remoção de Telemetrias Antigas (> 30 dias): (Tempo: {tempo_execucao:.4f} segundos)")
print(f"Leituras excluídas: {deleted_leituras_count_antigas}")

# Remove todos os veículos com tipo_propulsao: "Combustao" e suas telemetrias associadas. (Simulando fim do suporte para esse tipo de veículo)
inicio = time.time()
veiculo_ids_combustao = veiculos_col.distinct("_id", {"tipo_propulsao": "Combustao"})

res_veiculos_combustao = veiculos_col.delete_many({"_id": {"$in": veiculo_ids_combustao}})
deleted_veiculos_count_combustao = res_veiculos_combustao.deleted_count

res_leituras_combustao = leituras_col.delete_many({"id_veiculo": {"$in": veiculo_ids_combustao}})
deleted_leituras_count_combustao = res_leituras_combustao.deleted_count

fim = time.time()
tempo_execucao = fim - inicio
print(f"Remoção de frota à combustão: (Tempo: {tempo_execucao:.4f} segundos)")
print(f"Veículos excluídos: {deleted_veiculos_count_combustao}")
print(f"Telemetrias excluídas: {deleted_leituras_count_combustao}")

Remoção de Telemetrias Antigas (> 30 dias): (Tempo: 2.5178 segundos)
Leituras excluídas: 45815
Remoção de frota à combustão: (Tempo: 8.4845 segundos)
Veículos excluídos: 1664
Telemetrias excluídas: 1351


Etapa de remoção MySQL

In [ ]:
# Remove as leituras feitas a mais de 30 dias na coleção telemetrias. (Simulando uma política de retenção de dados periódicos)
inicio = time.time()

query_sql_1 = """
DELETE FROM Telemetrias
WHERE timestamp < DATE_SUB(NOW(), INTERVAL 30 DAY);
"""
mycursor.execute(query_sql_1)
deleted_leituras_count_antigas = mycursor.rowcount
mysqldb.commit()

fim = time.time()
tempo_execucao = fim - inicio
print(f"Remoção de Telemetrias Antigas (> 30 dias): (Tempo: {tempo_execucao:.4f} segundos)")
print(f"Leituras excluídas: {deleted_leituras_count_antigas}")

# Remove todos os veículos com tipo_propulsao: "Combustao" e suas telemetrias associadas. (Simulando fim do suporte para esse tipo de veículo)
inicio = time.time()

# (Graças ao ON DELETE CASCADE, basta deletar o veículo na tabela pai)
query_sql_2 = """
DELETE FROM Veiculos
WHERE tipo_propulsao = 'Combustao';
"""
mycursor.execute(query_sql_2)
deleted_veiculos_count_combustao = mycursor.rowcount
mysqldb.commit()

fim = time.time()
tempo_execucao = fim - inicio
print(f"Remoção de frota à combustão: (Tempo: {tempo_execucao:.4f} segundos)")
print(f"Veículos excluídos: {deleted_veiculos_count_combustao}")
print(f"Telemetrias excluídas: {"Devido à cascata, não é possível visualizar diretamente."}")

Remoção de Telemetrias Antigas (> 30 dias): (Tempo: 1.1335 segundos)
Leituras excluídas: 45790
Remoção de frota à combustão: (Tempo: 0.4740 segundos)
Veículos excluídos: 1664
Telemetrias excluídas: Devido à cascata, não é possível visualizar diretamente.
